In [9]:
!pip install transformers datasets --quiet

In [10]:
from datasets import load_dataset

# Load the GSM8K dataset
dataset = load_dataset("gsm8k", "main")

# Select the first 10 prompts for testing
samples = dataset["train"].select(range(500))

# Preview one example
for example in samples:
    print("Prompt:", example["question"])
    break


Prompt: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?


In [11]:
from transformers import BertTokenizer, BertForMaskedLM

# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval();  # Set model to inference mode


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
import torch
import torch.nn.functional as F

def compute_entropy(logits):
    probs = F.softmax(logits, dim=-1)
    log_probs = torch.log(probs + 1e-8)  # avoid log(0)
    entropy = -torch.sum(probs * log_probs, dim=-1)  # shape: [batch, seq_len]
    return entropy.mean(dim=-1).item()  # average entropy over sequence


In [13]:
results = []

for item in samples:
    prompt = item["question"]

    # Tokenize input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # shape: [1, seq_len, vocab]

    # Compute average entropy
    entropy_score = compute_entropy(logits)

    results.append({
        "prompt": prompt,
        "entropy_score": entropy_score
    })

# Show a few results
for r in results:
    print(f"Entropy: {r['entropy_score']:.4f} | Prompt: {r['prompt'][:60]}...")


Entropy: 0.5854 | Prompt: Natalia sold clips to 48 of her friends in April, and then s...
Entropy: 0.6240 | Prompt: Weng earns $12 an hour for babysitting. Yesterday, she just ...
Entropy: 0.3385 | Prompt: Betty is saving money for a new wallet which costs $100. Bet...
Entropy: 0.3097 | Prompt: Julie is reading a 120-page book. Yesterday, she was able to...
Entropy: 0.6913 | Prompt: James writes a 3-page letter to 2 different friends twice a ...
Entropy: 0.2681 | Prompt: Mark has a garden with flowers. He planted plants of three d...
Entropy: 0.3180 | Prompt: Albert is wondering how much pizza he can eat in one day. He...
Entropy: 0.2637 | Prompt: Ken created a care package to send to his brother, who was a...
Entropy: 0.1674 | Prompt: Alexis is applying for a new job and bought a new set of bus...
Entropy: 0.3392 | Prompt: Tina makes $18.00 an hour.  If she works more than 8 hours p...
Entropy: 0.2472 | Prompt: A deep-sea monster rises from the waters once every hundred ...
Entropy: 0

In [14]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("gsm8k_prompt_entropy_scores.csv", index=False)
df.head()


,prompt,entropy_score
0,Natalia sold clips to 48 of her friends in Apr...,0.585424
1,Weng earns $12 an hour for babysitting. Yester...,0.624047
2,Betty is saving money for a new wallet which c...,0.338496
3,"Julie is reading a 120-page book. Yesterday, s...",0.309732
4,James writes a 3-page letter to 2 different fr...,0.691290
